In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook



import os as os

import matplotlib.pyplot as plt

def imshow(img, cmap=None):
    plt.close('all')
    plt.figure()
    plt.imshow(img, cmap=cmap)
    plt.show()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inverse-problems-part-2" data-toc-modified-id="Inverse-problems-part-2-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Inverse problems part 2</a></span><ul class="toc-item"><li><span><a href="#Half-quadratic-splitting-method" data-toc-modified-id="Half-quadratic-splitting-method-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Half-quadratic splitting method</a></span></li><li><span><a href="#Alternating-direction-method-of-multipliers" data-toc-modified-id="Alternating-direction-method-of-multipliers-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Alternating direction method of multipliers</a></span></li></ul></li></ul></div>

# Inverse problems part 2

In this exercise we will try to implement some of the more advanced deconvolution algorithms on our own.


Test your filter implementations for different convolution kernels and apply different amounts of random noise to the convolution result before applying the deconvolution algorithms.

## Half-quadratic splitting method

Implement the HQS method for deconvolution and try different regularizers:
* Promote smoothness (i.e., minimize $\left\| \mathbf{x}  \right\|^2_2$),
* promote sparseness (i.e., minimize $\left\| \mathbf{x}  \right\|_1$),
* anisotropic total variation,
* isotropic total variation.

## Alternating direction method of multipliers

Implement the ADMM method for deconvolution and also try the above regularizers.